In [1]:
import dataextraction as de
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re,string,unicodedata
from string import punctuation
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rohang62/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
data = de.extract_data('data/train.jsonl')
train = de.parse_json(data, True)

data = de.extract_data('data/test.jsonl')
test = de.parse_json(data, False)

In [31]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
punctuation.append('@')
user = "USER"
stop.update(punctuation)
stop.update(user)

In [35]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i != "@USER":
            final_text.append(i.strip())
    return " ".join(final_text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

train['response']=train['response'].apply(denoise_text)
test['response']=test['response'].apply(denoise_text)
train['context']=train['context'].apply(denoise_text)
test['context']=test['context'].apply(denoise_text)

In [36]:
train.head()

,label,response,context
0,SARCASM,get .. obviously care would've moved right alo...,minor child deserves privacy kept politics Pam...
0,SARCASM,trying protest Talking labels label WTF make em,loser He's Press Secretary make excuses crowd ...
0,SARCASM,makes insane money MOVIES Einstein #LearnHowTh...,Donald J Trump guilty charged evidence clear S...
0,SARCASM,Meanwhile Trump even release SAT scores Wharto...,Jamie Raskin tanked Doug Collins Collins looks...
0,SARCASM,Pretty Sure Anti-Lincoln Crowd Claimed Democra...,Man ... ’ gone “ sides ” apocalypse one day al...


In [90]:
with open('train_ft.txt', 'w') as f:
    for each_text, each_label, each_context in zip(train['response'], train['label'], train['context']):
        f.writelines(f'__label__{each_label} {each_text} -> {each_context}\n')

In [130]:
import fasttext
model = fasttext.train_supervised('train_ft.txt', epoch=30, lr=0.05, wordNgrams=2, verbose=2, minCount=3)

In [131]:
def print_results(sample_size, precision, recall):
    precision   = round(precision, 2)
    recall      = round(recall, 2)
    print(f'{precision}')
    print(f'{recall}')

print_results(*model.test('train_ft.txt'))

0.96
0.96


In [132]:
def predict_is_sarcastic(text):
    return model.predict(text, k=2)

In [133]:
!rm answer.txt

In [134]:
with open('answer.txt', 'w') as f:
    for each_id, each_text, each_context in zip(test['id'], test['response'], test['context']):
        labels, probs = predict_is_sarcastic(f'{each_text} -> {each_context}')
        l = None
        if (probs[0] > probs[1] and labels[0] == "__label__SARCASM"):
            l = "SARCASM"
        elif (probs[0] > probs[1] and labels[0] != "__label__SARCASM"): 
            l = "NOT_SARCASM"
        elif (probs[0] < probs[1] and labels[0] == "__label__SARCASM"): 
            l = "NOT_SARCASM"
        else:
            l = "SARCASM"
        f.writelines(f'{each_id},{l}\n')